In [ ]:
import tensorflow as tf
import time
import tensorflow_hub as hub
import cv2
from PIL import Image
import uuid

from EfficientDet import *
import os
from glob import glob

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 10)])

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

# test_input에 대해서 object detection 수행

## pretrained model 가져오기

In [ ]:
model = get_detector() # daemon으로 올려서 singleton으로

In [ ]:
dataset_path = "./original_test/"
output_path = "./detected_data/detected_from_test/"
dataset_list = os.listdir(dataset_path)

In [ ]:
detected_objectList = object_detection(model, dataset_list,dataset_path, output_path) # detection에 약 8초


# retrieval model build

In [ ]:
from ImageRetrievalClass import ImageRetrievalClass
retrieval = ImageRetrievalClass("IncepResNet", True, False)
retrieval.buildModel()

# select one object 

In [ ]:
selectedObject = detected_objectList[1]

selectedObject_path = selectedObject["objectImagePath"]
selectedObject_pil = Image.open(selectedObject_path)
selectedObject_pil

In [ ]:
retrieval.readTestSet(selectedObject_pil)

In [ ]:
X_test = retrieval.testTransform()


In [ ]:
E_test = retrieval.predictTest(X_test)
E_test_flatten = E_test.reshape((-1, np.prod(retrieval.output_shape_model)))

# 동일 태그를 가진 객체들을 검색대상 pool에서 가져오기
- 추후 mongoDB 쿼리부분으로 바뀌어야 함

In [ ]:
selectedObject

In [ ]:
jsonFiles_path = glob("./jsonFiles/*.json")

In [ ]:
import json
queryed_json_pathList = list()
queryed_jsonList = list() # list whose element has same tag with selectedObject
for path in jsonFiles_path:
    with open(path, "rb") as f:
        jsonFile = json.load(f)
    
    if jsonFile["tag"] == selectedObject["tag"]:
        queryed_json_pathList.append(path)
        queryed_jsonList.append(jsonFile)

In [ ]:
queryed_jsonList[0]

# 태그로 쿼리한 json 파일의 npy(E_train)과 E_test를 비교

## json에서 npy가져와서 E_train 만들기

In [ ]:
E_train = list()
for json in queryed_jsonList:
    npy = np.load(json["npyPath"])
    E_train.append(npy)


In [ ]:
E_train = np.array(E_train)


In [ ]:
E_train_flatten = E_train.reshape((-1, np.prod(retrieval.output_shape_model)))
E_test_flatten.shape

In [ ]:
# E_train_flatten =np.load("E_train_flatten.npy")

In [ ]:
E_train_flatten.shape

In [ ]:
calculator = retrieval.similarityCalculator(E_train_flatten)

## 쿼리 결과의 pool 만들기

In [ ]:
retrieval_imagePool = [Image.open(json["objectImagePath"]) for json in queryed_jsonList]



## Retrieval

In [ ]:
retrieval.retrieval(E_test_flatten=E_test_flatten, calculator=calculator, retrieval_imagePool=retrieval_imagePool )

E_test_flatten.shape

In [ ]:
# 원본 이미지 